# Exercise 16


In [92]:
import pandas as pd
import numpy as np
train_df = pd.read_csv('14_mashable_train_df.csv', index_col=0)
train_df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,Popular
0,http://mashable.com/2013/09/16/sprints-upgrade...,479,11,1045,0.407946,1,0.617021,29,3,1,...,0.033333,1.0,-0.305139,-0.700000,-0.125000,0.0,0.0,0.5,0.0,1
1,http://mashable.com/2014/12/06/eric-garner-pro...,31,14,0,0.000000,0,0.000000,0,0,0,...,0.000000,0.0,0.000000,0.000000,0.000000,1.0,-0.5,0.5,0.5,0
2,http://mashable.com/2013/06/06/human-scale-inv...,581,6,604,0.470000,1,0.675516,9,0,1,...,0.100000,1.0,-0.215873,-0.666667,-0.071429,0.1,0.0,0.4,0.0,0
3,http://mashable.com/2014/11/24/beyonce-spotify...,44,16,708,0.474286,1,0.675325,5,2,2,...,0.062500,0.5,-0.376852,-0.875000,-0.100000,0.4,-0.2,0.1,0.2,0
4,http://mashable.com/2013/03/03/superheroic-let...,676,7,30,1.000000,1,1.000000,2,1,0,...,0.250000,0.7,0.000000,0.000000,0.000000,0.0,0.0,0.5,0.0,1


In [93]:
train_df.shape

(23786, 61)

In [94]:
X = train_df.drop(['url', 'Popular'], axis=1)
y = train_df['Popular'].values

# Exercice 16.1 (2 points)

Estimate a Logistic Regression, GaussianNB, K-nearest neighbors and a Decision Tree **Classifiers**

Evaluate using the following metrics:
* Accuracy
* F1-Score
* F_Beta-Score (Beta=2)

Comment about the results

Combine the classifiers and comment

In [95]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

models = {'lr': LogisticRegression(),
          'nb': GaussianNB(),
          'nn': KNeighborsClassifier(),
          'dt': DecisionTreeClassifier()}

In [96]:
X = train_df.drop(['url', 'Popular'], axis=1)
y = train_df['Popular'].values
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)

In [97]:
for model in models.keys():
    models[model].fit(X_train, y_train)
y_pred = pd.DataFrame(index=X_test.index, columns=models.keys())
for model in models.keys():
    y_pred[model] = models[model].predict(X_test)

In [101]:
from sklearn.metrics import accuracy_score
for model in models.keys():
    print(model, 'Accuracy', accuracy_score(y_pred[model],y_test))    

nn Accuracy 0.857070791996
lr Accuracy 0.871363712796
dt Accuracy 0.774340003363
nb Accuracy 0.839246678998


In [99]:
from sklearn.metrics import f1_score
for model in models.keys():
    print(model, 'F1_Score',f1_score(y_pred[model],y_test))

nn F1_Score 0.0597345132743
lr F1_Score 0.0103492884864
dt F1_Score 0.190591073583
nb F1_Score 0.153982300885


In [100]:
from sklearn.metrics import fbeta_score
for model in models.keys():
    print(model, 'F_Beta_Score',fbeta_score(y_pred[model],y_test,beta=2))

nn F_Beta_Score 0.100596125186
lr F_Beta_Score 0.0244498777506
dt F_Beta_Score 0.181275814594
nb F_Beta_Score 0.193677649154


Se observa que todos los modelos presentan buen accuracy siendo la regresión logistica la que mejor ajuste presenta y el arbol de clasificación el de menor, sin embargo en cuanto al F1 score y el Fbeta score es bajo, dado que estos modelos presentan una presición baja en cuanto al porcentaje de resultados positivos obtenidos.

# Exercice 16.2

Apply random-undersampling with a target percentage of 0.5

how does the results change

In [65]:
def UnderSampling(X_train, y_train, target_percentage=0.5, seed=None):
    # Assuming minority class is the positive
    n_samples = y_train.shape[0]
    n_samples_0 = (y_train == 0).sum()
    n_samples_1 = (y_train == 1).sum()

    n_samples_0_new =  n_samples_1 / target_percentage - n_samples_1
    n_samples_0_new_per = n_samples_0_new / n_samples_0

    filter_ = y_train == 0

    np.random.seed(seed)
    rand_1 = np.random.binomial(n=1, p=n_samples_0_new_per, size=n_samples)
    
    filter_ = filter_ & rand_1
    filter_ = filter_ | (y_train == 1)
    filter_ = filter_.astype(bool)
    
    return X_train[filter_], y_train[filter_]

In [66]:
X_u, y_u=UnderSampling(X_train, y_train, 0.5, 123)

for model in models.keys():
    models[model].fit(X_u, y_u)
    
# predict test for each model
y_pred = pd.DataFrame(index=X_test.index, columns=models.keys())
for model in models.keys():
    y_pred[model] = models[model].predict(X_test)

In [67]:
from sklearn.metrics import accuracy_score, f1_score, fbeta_score
for model in models.keys():
    print(model, 'Accuracy',accuracy_score(y_pred[model],y_test))

nn Accuracy 0.581469648562
lr Accuracy 0.660669244997
dt Accuracy 0.571212375988
nb Accuracy 0.823608542122


In [68]:
from sklearn.metrics import accuracy_score, f1_score, fbeta_score
for model in models.keys():
    print(model, 'F1_Score',f1_score(y_pred[model],y_test))

nn F1_Score 0.249170437406
lr F1_Score 0.302213001383
dt F1_Score 0.250440917108
nb F1_Score 0.196168582375


In [69]:
from sklearn.metrics import accuracy_score, f1_score, fbeta_score
for model in models.keys():
    print(model, 'F_Beta_Score',fbeta_score(y_pred[model],y_test,beta=2))

nn F_Beta_Score 0.187966502822
lr F_Beta_Score 0.235097912632
dt F_Beta_Score 0.187930121758
nb F_Beta_Score 0.217243720299


Se observa que al balancear la base de datos mediante un muetreo sobre las categorias que presentan mayor ocurrencia, es decir sobre las calificaciones del articulo como no popular (0), se obtiene que todos los modelos anteriormente estimados presentan  una disminución en su accuracy debido a que estos estaban prediciendo erradamente los articulos populares, sin embargo se tiene que al realizar este balanceo el F1_Score y el F_Beta_Score aumentan con relación a las estimaciones que se obtuvieron con la base de datos desbalanceada.

# Exercice 16.3 (3 points)

For each model estimate a BaggingClassifier of 100 models using the under sampled datasets

In [70]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(max_depth=None,random_state=42)

In [71]:
from sklearn.ensemble import BaggingClassifier
bagreg = BaggingClassifier(DecisionTreeClassifier(), n_estimators=100, 
                          bootstrap=True, oob_score=True, random_state=1)

In [72]:
bagreg.fit(X_u, y_u)
y_pred = bagreg.predict(X_test)

In [73]:
from sklearn.metrics import accuracy_score, f1_score, fbeta_score
Accuracy = accuracy_score(y_pred, y_test)
F1_Score= f1_score(y_pred, y_test)
F_Beta_Score = fbeta_score(y_pred, y_test,beta=2)

In [74]:
Accuracy, F1_Score ,F_Beta_Score

(0.64385404405582647, 0.31633311814073595, 0.24214271595176912)

# Exercice 16.4 (2 points)

Using the under-sampled dataset

Evaluate a RandomForestClassifier and compare the results

change n_estimators=100, what happened

In [75]:
from sklearn.ensemble import RandomForestClassifier
rfcla = RandomForestClassifier(n_estimators=150, oob_score=True, random_state=1, n_jobs=-1)
rfcla

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=-1,
            oob_score=True, random_state=1, verbose=0, warm_start=False)

In [76]:
rfcla.fit(X_u, y_u)
y_pred = rfcla.predict(X_test)

In [77]:
from sklearn.metrics import accuracy_score, f1_score, fbeta_score
Accuracy = accuracy_score(y_pred, y_test)
F1_Score= f1_score(y_pred, y_test)
F_Beta_Score = fbeta_score(y_pred, y_test,beta=2)

In [78]:
Accuracy, F1_Score ,F_Beta_Score

(0.65041197242307047, 0.32827140549273021, 0.2513358400949931)

In [79]:
from sklearn.ensemble import RandomForestClassifier
rfcla = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=1, n_jobs=-1)
rfcla

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=True, random_state=1, verbose=0, warm_start=False)

In [80]:
rfcla.fit(X_u, y_u)
y_pred = rfcla.predict(X_test)

In [81]:
from sklearn.metrics import accuracy_score, f1_score, fbeta_score
Accuracy = accuracy_score(y_pred, y_test)
F1_Score= f1_score(y_pred, y_test)
F_Beta_Score = fbeta_score(y_pred, y_test,beta=2)

In [82]:
Accuracy, F1_Score ,F_Beta_Score

(0.6488986043383218, 0.32251784555483448, 0.24716530734036204)

Se observa que las predicciones realizadas mediante randonforest presenta mayor presición con relación a los modelos que se ajustaron inicialmente (nn, lr, nb, dt), y se observa que al disminuir la cantidad de arboles de 150 a 100 el ajuste disminuye levemente en su Accuracy, F1_Score y F_Beta_Score por lo que se puede afirmar que entre mayores combinaciones de arboles se realicen sus predicciones aumentaran en su presición.